In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import *
import statsmodels.api as sm
fair_df = pd.read_csv('data/fair.csv')
def draw_line(slope, intercept):
    def f(x):
        return intercept*(slope-1)/30*x +intercept
    x = np.arange(-15,15)
    y = f(x)
    points = (zip(fair_df.GROWTH, fair_df.VOTE))
    print("Square Residual Sum:", sum([(y-f(x))**2 for x,y in points]))
    print("Absolute Residual Sum:", sum([abs(y-f(x))for x, y in points]))
    plt.plot(x,y)
    sns.scatterplot('GROWTH', 'VOTE', data=fair_df, color='black')
    plt.xlabel('GDP Growth per Capita')
    
interact(draw_line, slope=(0.0,3), intercept=(30,70))

# (-15, intercept) (15, intercept*slope) slope= intercept*(slope-1)/30

interactive(children=(FloatSlider(value=1.5, description='slope', max=3.0), IntSlider(value=50, description='i…

<function __main__.draw_line(slope, intercept)>

In [47]:
sm.ols('VOTE ~ GROWTH+INFLATION', data=fair_df).fit().summary()

AttributeError: module 'statsmodels.api' has no attribute 'ols'